In [ ]:
import json

# Example JSON-like data
data = {
    "4": [
        {
            "rx_packets": 802,
            "tx_packets": 884,
            "rx_bytes": 87282,
            "tx_bytes": 97735,
            "rx_dropped": 0,
            "tx_dropped": 0,
            "rx_errors": 0,
            "tx_errors": 0,
            "rx_frame_err": 0,
            "rx_over_err": 0,
            "rx_crc_err": 0,
            "collisions": 0,
            "duration_sec": 78,
            "duration_nsec": 512000000,
            "port_no": 5
        }
    ]
}

# Retrieve tx_packets
tx_packets = None
for key in data:
    if "tx_packets" in data[key][0]:
        tx_packets = data[key][0]["tx_packets"]
        break

print("tx_packets:", tx_packets)

In [11]:

import requests

# Global variables
DPID = 4 # central switch ID
PORT_TO_GWI = 5 # port to GWI
THRESHOLD = 20000 # threshold for latency
CHECK_INTERVAL = 2 #
stats= requests.get(f"http://localhost:8080/stats/port/{DPID}/{PORT_TO_GWI}").json()
for port_stats in stats[str(DPID)]:
    print(port_stats)
    if port_stats['port_no'] == PORT_TO_GWI:
        print("Oui")
        print(port_stats.get('tx_packets', 0))

{'rx_packets': 24327, 'tx_packets': 24548, 'rx_bytes': 2685698, 'tx_bytes': 2731958, 'rx_dropped': 0, 'tx_dropped': 0, 'rx_errors': 0, 'tx_errors': 0, 'rx_frame_err': 0, 'rx_over_err': 0, 'rx_crc_err': 0, 'collisions': 0, 'duration_sec': 2656, 'duration_nsec': 423000000, 'port_no': 5}
Oui
24548


In [ ]:
import requests

# Define the flow entry payload
redirection_rule = {
    "dpid": 4,
    "priority": 100,
    "match": {
        "in_port": 1,
        "ipv4_src": "10.0.0.3",
        "ipv4_dst": "10.0.0.2",
        "eth_type": 2048
    },
    "actions": [
        {
            "type": "OUTPUT",
            "port": 4
        }
    ]
}

# Send the POST request
try:
    response = requests.post("http://localhost:8080/stats/flowentry/add", json=redirection_rule)
    print("Status Code:", response.status_code)
    print("Response Text:", response.text)
except requests.RequestException as e:
    print("Error sending POST request:", e)

Status Code: 200
Response Text: 


In [20]:
import subprocess
def execute():
    """
    Execute function to apply the planned actions.
    """
    print("[Executing actions...] Starting gateway.js in GI2 and redirecting traffic to GI2 via ofctl_rest.")
    
    # Start gateway.js in the gi2 container
    try:
        command = (
            "docker exec mn.gi2 nohup "
            "node gateway.js "
            "--local_ip 10.0.0.12 "
            "--local_port 8182 "
            "--local_name gi2 "
            "--remote_ip 10.0.0.1 "
            "--remote_port 8080 "
            "--remote_name srv > /tmp/gi2.log 2>&1 &"
        )
        subprocess.Popen(command, shell =True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("[INFO] gateway.js started successfully in the gi2 container.")
    except Exception as e:
        print("[ERROR] Failed to start gateway.js in gi2 container:", e)

In [22]:
execute()

[Executing actions...] Starting gateway.js in GI2 and redirecting traffic to GI2 via ofctl_rest.
[INFO] gateway.js started successfully in the gi2 container.


In [ ]:
curl -X POST -d '{
     "table_id": 0,
     "out_port": 2,
     "cookie": 1,
     "cookie_mask": 1,
     "match":{
         "in_port":1
     }
}' http://localhost:8080/stats/flow/1

SyntaxError: invalid syntax (2225002556.py, line 1)